## Behavior ISC: AnnaK Method

In [ ]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
from scipy.stats import rankdata
import seaborn as sns
import matplotlib.pyplot as plt
from jupyterthemes import jtplot
jtplot.style(theme='grade3') 
from Function import get_tril_vec

# define data repository
data_dir = 'F:/1_Emotion_Data/Data/Behavior/Data/Social_Scale/'
results_dir = 'F:/1_Emotion_Data/Results/0_ISC/Behav/'

### Calculate the total score of Scale

Neuroticism scale

In [ ]:
ne = pd.read_csv(os.path.join(data_dir,'EM.csv'))
tp = ne.iloc[:,2:]
ne_score = tp.sum(axis=1)
ne['sum_score'] = ne_score
ne.to_csv(os.path.join(data_dir, 'EM.csv'))

Interactive Mentalizing questionaire
* Self-other: 3, 4, 5, 7, 8, 10
* Self-self: 2, 11, 14, 15, 16, 17, 19, 20
* Other-self: 1, 6, 9, 12, 13, 18

In [ ]:
imq = pd.read_csv(os.path.join(data_dir, 'IMQ.csv'))
so = imq['IMQ_3'] + imq['IMQ_4'] + imq['IMQ_5'] + imq['IMQ_7'] + imq['IMQ_8'] + imq['IMQ_10']
ss = imq['IMQ_2'] + imq['IMQ_11'] + imq['IMQ_14'] + imq['IMQ_15'] + imq['IMQ_16'] + imq['IMQ_17'] + imq['IMQ_19'] + imq['IMQ_20']
otherself = imq['IMQ_1'] + imq['IMQ_6'] + imq['IMQ_9'] + imq['IMQ_12'] + imq['IMQ_13'] + imq['IMQ_18']
imq['SO'] = so
imq['SS'] = ss
imq['OS'] = otherself
imq.to_csv(os.path.join(data_dir, 'IMQ.csv'))

In [ ]:
for i in ('ag','ax', 'fe', 'hl','ha'):
    imq = pd.read_csv(os.path.join(data_dir, i + '/' +i + '_IMQ.csv'))
    so = imq['IMQ_3'] + imq['IMQ_4'] + imq['IMQ_5'] + imq['IMQ_7'] + imq['IMQ_8'] + imq['IMQ_10']
    ss = imq['IMQ_2'] + imq['IMQ_11'] + imq['IMQ_14'] + imq['IMQ_15'] + imq['IMQ_16'] + imq['IMQ_17'] + imq['IMQ_19'] + imq['IMQ_20']
    imq['SO'] = so
    imq['SS'] = ss
    imq.to_csv(os.path.join(data_dir, i + '/' +i + '_IMQ.csv'))
    del imq

 Inerpersonal reactivity index
* Fantasy: 1, 4, 6, 11, 15, 22, 25
* Empathic concern: 2, 8, 13, 17, 19, 21
* Personal distress: 5, 9, 12, 16, 18, 23, 26
* Perspective taking: 3, 7, 10, 14, 20, 24, 27

In [ ]:
iri = pd.read_csv(os.path.join(data_dir,'IRI.csv'))
fs = iri['IRI_1'] + iri['IRI_4'] + iri['IRI_6'] + iri['IRI_11'] + iri['IRI_15'] + iri['IRI_22'] + iri['IRI_25']
ec = iri['IRI_2'] + iri['IRI_8'] + iri['IRI_13'] + iri['IRI_17'] + iri['IRI_19'] + iri['IRI_21']
pds = iri['IRI_5'] + iri['IRI_9'] + iri['IRI_12'] + iri['IRI_16'] + iri['IRI_18'] + iri['IRI_23'] + iri['IRI_26']
pt = iri['IRI_3'] + iri['IRI_7'] + iri['IRI_10'] + iri['IRI_14'] + iri['IRI_20'] + iri['IRI_24'] + iri['IRI_27']
iri['FS'] = fs
iri['EC'] = ec
iri['PD'] = pds
iri['PT'] = pt
iri.to_csv(os.path.join(data_dir, 'IRI.csv'))

In [ ]:
for i in ('ag','ax', 'fe', 'hl','ha'):
    iri = pd.read_csv(os.path.join(data_dir, i + '/' +i + '_IRI.csv'))
    fs = iri['IRI_1'] + iri['IRI_4'] + iri['IRI_6'] + iri['IRI_11'] + iri['IRI_15'] + iri['IRI_22'] + iri['IRI_25']
    ec = iri['IRI_2'] + iri['IRI_8'] + iri['IRI_13'] + iri['IRI_17'] + iri['IRI_19'] + iri['IRI_21']
    pds = iri['IRI_5'] + iri['IRI_9'] + iri['IRI_12'] + iri['IRI_16'] + iri['IRI_18'] + iri['IRI_23'] + iri['IRI_26']
    iri['FS'] = fs
    iri['EC'] = ec
    iri['PD'] = pds
    iri.to_csv(os.path.join(data_dir, i + '/' +i + '_IRI.csv'))
    del iri

Summary score correlation

In [ ]:
summary = pd.concat([imq['SO'], imq['SS'], imq['OS'], iri['EC'], iri['PD'], iri['FS'], iri['PT']], axis=1)

In [ ]:
corr = summary.corr()

In [ ]:
sns.heatmap(corr, square=True, cmap='Reds', linewidths=0.1, xticklabels=True, yticklabels=True, annot=True)

In [ ]:
import plotly.express as px
fig = px.scatter_3d(summary, x='SO', y='SS', z='EC')
fig.show()

### Secion 1: Angry (IMQ-SO/SS, IRI-EC/PD)

* Angry IMQ-SO/SS

In [ ]:
def imq_annak(n_subs, raw_data):
    so,ss = raw_data['SO'], raw_data['SS']
    so_rank, ss_rank = rankdata(so), rankdata(ss) # explicity convert the raw scores to ranks
    simi_so, simi_ss = np.zeros((n_subs, n_subs)), np.zeros((n_subs, n_subs))
    
    for i in range(n_subs):
        for j in range(n_subs):
            if i < j:
                sim_ij = np.mean([so_rank[i], so_rank[j]])/n_subs
                simi_so[i,j] = sim_ij
                simi_so[j,i] = sim_ij
            elif i==j:
                simi_so[i,j] = 1
    for i in range(n_subs):
        for j in range(n_subs):
            if i < j:
                sim_ij = np.mean([ss_rank[i], ss_rank[j]])/n_subs
                simi_ss[i,j] = sim_ij
                simi_ss[j,i] = sim_ij
            elif i==j:
                simi_ss[i,j] = 1
    return simi_so, simi_ss

def iri_annak(n_subs, raw_data):
    ec,pd = raw_data['EC'], raw_data['PD']
    ec_rank, pd_rank = rankdata(ec), rankdata(pd) # explicity convert the raw scores to ranks
    simi_ec, simi_pd = np.zeros((n_subs, n_subs)), np.zeros((n_subs, n_subs))
    
    for i in range(n_subs):
        for j in range(n_subs):
            if i < j:
                sim_ij = np.mean([ec_rank[i], ec_rank[j]])/n_subs
                simi_ec[i,j] = sim_ij
                simi_ec[j,i] = sim_ij
            elif i==j:
                simi_ec[i,j] = 1
    for i in range(n_subs):
        for j in range(n_subs):
            if i < j:
                sim_ij = np.mean([pd_rank[i], pd_rank[j]])/n_subs
                simi_pd[i,j] = sim_ij
                simi_pd[j,i] = sim_ij
            elif i==j:
                simi_pd[i,j] = 1
    return simi_ec, simi_pd

In [ ]:
#Set the paprameters and import data
n_subs = 36
imq_raw = pd.read_csv(os.path.join(data_dir, 'ag/ag_IMQ.csv'))
so_simi, ss_simi = imq_annak(n_subs, imq_raw)

# Plot the isc matrix
mask =np.zeros_like(so_simi)
mask[np.triu_indices_from(mask)] = True
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(25,12))
sns.heatmap(so_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax1.set_title("Angry IMQ-SO ISC", fontsize=25, fontweight='bold')
sns.heatmap(ss_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax2, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax2.set_title("Angry IMQ-SS ISC", fontsize=25, fontweight='bold')

# Save the result
ag_simi_so = get_tril_vec(so_simi)
ag_simi_ss = get_tril_vec(ss_simi)

* Angry IRI-EC/PD

In [ ]:
# Set the paprameters and import data
n_subs = 36
iri_raw = pd.read_csv(os.path.join(data_dir, 'ag/ag_IRI.csv'))
ec_simi, pd_simi = iri_annak(n_subs, iri_raw)

# Plot the isc matrix
mask =np.zeros_like(ec_simi)
mask[np.triu_indices_from(mask)] = True
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(25,12))
sns.heatmap(ec_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax1.set_title("Angry IRI-EC ISC", fontsize=25, fontweight='bold')
sns.heatmap(pd_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax2, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax2.set_title("Angry IRI-PD ISC", fontsize=25, fontweight='bold')

# Save the result
ag_simi_ec = get_tril_vec(ec_simi)
ag_simi_pd = get_tril_vec(pd_simi)

### Section 2: Anxiety (IMQ-SO/SS, IRI-EC/PD)

Anxiety IMQ-SO/SS

In [ ]:
# Set the paprameters and import data
n_subs = 38
imq_raw = pd.read_csv(os.path.join(data_dir, 'ax/ax_IMQ.csv'))
so_simi, ss_simi = imq_annak(n_subs, imq_raw)

# Plot the isc matrix
mask =np.zeros_like(so_simi)
mask[np.triu_indices_from(mask)] = True
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(25,12))
sns.heatmap(so_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax1.set_title("Anxiety IMQ-SO ISC", fontsize=25, fontweight='bold')
sns.heatmap(ss_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax2, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax2.set_title("Anxiety IMQ-SS ISC", fontsize=25, fontweight='bold')

# Save the result
ax_simi_so = get_tril_vec(so_simi)
ax_simi_ss = get_tril_vec(ss_simi)

Anxiety IRI-EC/PD

In [ ]:
# Set the paprameters and import data
n_subs = 38
iri_raw = pd.read_csv(os.path.join(data_dir, 'ax/ax_IRI.csv'))
ec_simi, pd_simi = iri_annak(n_subs, iri_raw)

# Plot the isc matrix
mask =np.zeros_like(ec_simi)
mask[np.triu_indices_from(mask)] = True
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(25,12))
sns.heatmap(ec_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax1.set_title("Anxiety IRI-EC ISC", fontsize=25, fontweight='bold')
sns.heatmap(pd_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax2, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax2.set_title("Anxiety IRI-PD ISC", fontsize=25, fontweight='bold')

# Save the result
ax_simi_ec = get_tril_vec(ec_simi)
ax_simi_pd = get_tril_vec(pd_simi)

### Section 3: Fear (IMQ-SS, IRI-FS/PD)

Fear IMQ-SS/SO

In [ ]:
# Set the paprameters and import data
n_subs = 37
imq_raw = pd.read_csv(os.path.join(data_dir, 'fe/fe_IMQ.csv'))
so_simi, ss_simi = imq_annak(n_subs, imq_raw)

# Plot the isc matrix
mask =np.zeros_like(so_simi)
mask[np.triu_indices_from(mask)] = True
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(25,12))
sns.heatmap(so_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax1.set_title("Fear IMQ-SO ISC", fontsize=25, fontweight='bold')
sns.heatmap(ss_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax2, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax2.set_title("Fear IMQ-SS ISC", fontsize=25, fontweight='bold')

# Save the result
fe_simi_ss = get_tril_vec(so_simi)
fe_simi_so = get_tril_vec(ss_simi)

Fear IMQ-FS/EC/PD

In [ ]:
# Set the paprameters and import data
n_subs = 37
iri_raw = pd.read_csv(os.path.join(data_dir, 'fe/fe_IRI.csv'))
ec_simi, pd_simi = iri_annak(n_subs, iri_raw)

iri_fs = iri_raw['FS']
iri_fs_rank= rankdata(iri_fs) # explicity convert the raw scores to ranks
fs_simi= np.zeros((n_subs, n_subs))

# Calculate the ann isc of fs
for i in range(n_subs):
    for j in range(n_subs):
        if i < j:
            sim_ij = np.mean([iri_fs_rank[i], iri_fs_rank[j]])/n_subs
            fs_simi[i,j] = sim_ij
            fs_simi[j,i] = sim_ij
        elif i==j:
            fs_simi[i,j] = 1

# Plot the isc matrix
mask =np.zeros_like(ec_simi)
mask[np.triu_indices_from(mask)] = True
fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, figsize=(25,12))
sns.heatmap(ec_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax1.set_title("Helpless IRI-EC ISC", fontsize=25, fontweight='bold')
sns.heatmap(pd_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax2, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax2.set_title("Helpless IRI-PD ISC", fontsize=25, fontweight='bold')
sns.heatmap(fs_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax3, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax3.set_title("Helpless IRI-FS ISC", fontsize=25, fontweight='bold')

# Save the result
fe_simi_ec = get_tril_vec(ec_simi)
fe_simi_pd = get_tril_vec(pd_simi)
fe_simi_fs = get_tril_vec(fs_simi)

### Section 4: Helpless (IMQ-SO, IRI-EC/PD)

Helpless IMQ-SO

In [ ]:
# Set the paprameters and import data
n_subs = 39
imq_raw = pd.read_csv(os.path.join(data_dir, 'hl/hl_IMQ.csv'))
so_simi, ss_simi = imq_annak(n_subs, imq_raw)
# Plot the isc matrix
mask =np.zeros_like(so_simi)
mask[np.triu_indices_from(mask)] = True
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(25,12))
sns.heatmap(so_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax1.set_title("Helpless IMQ-SO ISC", fontsize=25, fontweight='bold')
sns.heatmap(ss_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax2, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax2.set_title("Helpless IMQ-SS ISC", fontsize=25, fontweight='bold')

# Save the result
hl_simi_ss = get_tril_vec(so_simi)
hl_simi_so = get_tril_vec(ss_simi)

Helpless IRI-EC/PD

In [ ]:
# Set the paprameters and import data
n_subs = 39
iri_raw = pd.read_csv(os.path.join(data_dir, 'hl/hl_IRI.csv'))
ec_simi, pd_simi = iri_annak(n_subs, iri_raw)

# Plot the isc matrix
mask =np.zeros_like(ec_simi)
mask[np.triu_indices_from(mask)] = True
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(25,12))
sns.heatmap(ec_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax1.set_title("Helpless IRI-EC ISC", fontsize=25, fontweight='bold')
sns.heatmap(pd_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax2, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax2.set_title("Helpless IRI-PD ISC", fontsize=25, fontweight='bold')

# Save the result
hl_simi_ec = get_tril_vec(ec_simi)
hl_simi_pd = get_tril_vec(pd_simi)

### Section 5: Happy (IMQ-SS/SO, IRI-FS)

Happy IMQ-SS/SO

In [ ]:
# Set the paprameters and import data
n_subs = 41
imq_raw = pd.read_csv(os.path.join(data_dir, 'ha/ha_IMQ.csv'))
so_simi, ss_simi = imq_annak(n_subs, imq_raw)
# Plot the isc matrix
mask =np.zeros_like(so_simi)
mask[np.triu_indices_from(mask)] = True
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(25,12))
sns.heatmap(so_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax1.set_title("Happy IMQ-SO ISC", fontsize=25, fontweight='bold')
sns.heatmap(ss_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax2, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax2.set_title("Happy IMQ-SS ISC", fontsize=25, fontweight='bold')

# Save the result
ha_simi_ss = get_tril_vec(so_simi)
ha_simi_so = get_tril_vec(ss_simi)

* Happy IRI-FS

In [ ]:
# Set the paprameters and import data
n_subs = 41
iri_raw = pd.read_csv(os.path.join(data_dir, 'ha/ha_IRI.csv'))
iri_fs = iri_raw['FS']
iri_fs_rank= rankdata(iri_fs) # explicity convert the raw scores to ranks
fs_simi= np.zeros((n_subs, n_subs))

# Calculate the ann isc
for i in range(n_subs):
    for j in range(n_subs):
        if i < j:
            sim_ij = np.mean([iri_fs_rank[i], iri_fs_rank[j]])/n_subs
            fs_simi[i,j] = sim_ij
            fs_simi[j,i] = sim_ij
        elif i==j:
            fs_simi[i,j] = 1

# Plot the isc matrix
mask =np.zeros_like(fs_simi)
mask[np.triu_indices_from(mask)] = True
plt.figure(figsize=(25, 12))
sns.heatmap(fs_simi, square=True, cmap='RdBu_r', linewidths=0.1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})

# Save the result
#ha_simi_fs = get_tril_vec(fs_simi)

In [ ]:
behavisc= {'ag_so':ag_simi_so, 'ag_ss':ag_simi_ss, 'ag_ec':ag_simi_ec, 'ag_pd':ag_simi_pd, 'ax_so':ax_simi_so, 'ax_ss':ax_simi_ss, 'ax_ec':ax_simi_ec, 'ax_pd':ax_simi_pd,\
    'fe_so':fe_simi_so, 'fe_ss':fe_simi_ss, 'fe_ec':fe_simi_ec, 'fe_pd':fe_simi_pd, 'fe_fs':fe_simi_fs, 'hl_so':hl_simi_so, 'hl_ss':hl_simi_ss, 'hl_ec':hl_simi_ec, 'hl_pd':hl_simi_pd,\
     'ha_so':ha_simi_so, 'ha_ss':ha_simi_ss, 'ha_fs':ha_simi_fs}
np.save(os.path.join(results_dir, 'behavisc_annak.npy'), behavisc) 